In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split

In [2]:
df = pd.read_csv('events.csv')
print(f"{df.shape}")

(2756101, 5)


In [3]:
df.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [4]:
df.rename(columns={'visitorid': 'UserId', 'itemid': 'ProductId', 'timestamp': 'Timestamp'}, inplace=True)


In [5]:
df = df[df['event'].isin(['view', 'addtocart', 'transaction'])]
df.head()

,Timestamp,UserId,event,ProductId,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [6]:
reward_map = {'view': 0.2, 'addtocart': 0.5, 'transaction': 1.0}
df['reward'] = df['event'].map(reward_map)

In [7]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')
df.head()

,Timestamp,UserId,event,ProductId,transactionid,reward
0,2015-06-02 05:02:12.117,257597,view,355908,NaN,0.2
1,2015-06-02 05:50:14.164,992329,view,248676,NaN,0.2
2,2015-06-02 05:13:19.827,111016,view,318965,NaN,0.2
3,2015-06-02 05:12:35.914,483717,view,253185,NaN,0.2
4,2015-06-02 05:02:17.106,951259,view,367447,NaN,0.2


In [8]:
user_counts = df['UserId'].value_counts()
frequent_users = user_counts[user_counts >= 6].index
df = df[df['UserId'].isin(frequent_users)]
print(f"Data after filtering: {df.shape}")

Data after filtering: (833702, 6)


In [9]:
item_counts = df['ProductId'].value_counts()
frequent_items = item_counts[item_counts >= 6].index
df = df[df['ProductId'].isin(frequent_items)]
print(f"Data after filtering: {df.shape}")

Data after filtering: (707162, 6)


In [10]:
user_encoder = LabelEncoder()
df['UserId'] = user_encoder.fit_transform(df['UserId']) + 1

In [11]:
item_encoder = LabelEncoder()
df['ProductId'] = item_encoder.fit_transform(df['ProductId']) + 1

In [12]:
num_items = df['ProductId'].max() + 1  
print(f"Total number of items (including padding index): {num_items}")

Total number of items (including padding index): 31228


In [13]:
df = df.sort_values(['UserId', 'Timestamp']).reset_index(drop=True)
df.head()

,Timestamp,UserId,event,ProductId,transactionid,reward
0,2015-08-07 17:51:44.567,1,view,21761,NaN,0.2
1,2015-08-07 17:53:33.790,1,view,21761,NaN,0.2
2,2015-08-07 17:56:52.664,1,view,17456,NaN,0.2
3,2015-08-07 18:01:08.920,1,view,14493,NaN,0.2
4,2015-08-07 18:08:25.669,1,view,22900,NaN,0.2


In [14]:
if (df['UserId'] == 0).any():
    print("The encoder assigned 0 as a value for some UserIdUserId entries.")
else:
    print("No UserId entry has been assigned the value 0.")

if (df['ProductId'] == 0).any():
    print("The encoder assigned 0 as a value for some ProductId entries.")
else:
    print("No ProductIdProductId entry has been assigned the value 0.")



No UserId entry has been assigned the value 0.
No ProductIdProductId entry has been assigned the value 0.


In [15]:
user_sequences = df.groupby('UserId')['ProductId'].apply(list).values


In [16]:
def generate_sequences(sequence, window_size=10):
    if len(sequence) < window_size:
        sequence = [0] * (window_size - len(sequence)) + sequence  # Pad shorter sequences
    return [sequence[i:i + window_size] for i in range(len(sequence) - window_size + 1)]

In [17]:
all_sequences = []
for seq in user_sequences:
    all_sequences.extend(generate_sequences(seq))

In [18]:
all_sequences = torch.tensor(all_sequences, dtype=torch.long)
inputs = all_sequences[:, :-1]  
targets = all_sequences[:, -1]  

In [19]:
print(all_sequences[:5])

tensor([[    0,     0, 21761, 21761, 17456, 14493, 22900, 22900, 14493, 21761],
        [    0,     0,     0,     0,  4299, 17001, 23009, 23009, 23009, 23009],
        [    0,     0,     0, 11248, 11248, 11248, 11248, 11248, 11248, 11248],
        [    0,     0,     0,     0,     0,     0, 13291, 28742, 13291,  3297],
        [    0, 18993,  2553, 21410, 18993, 21410, 29604, 29604, 16722,  2553]])


In [20]:
print(f"Inputs shape: {inputs.shape}, Targets shape: {targets.shape}")


Inputs shape: torch.Size([341868, 9]), Targets shape: torch.Size([341868])


In [21]:
print(f"Max target value: {targets.max().item()}, Expected range: [0, {num_items - 1}]")
print(f"Min target value: {targets.min().item()}")


Max target value: 31227, Expected range: [0, 31227]
Min target value: 1


In [22]:
dataset = TensorDataset(inputs, targets)

total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Train size: {len(train_loader.dataset)}")
print(f"Validation size: {len(val_loader.dataset)}")
print(f"Test size: {len(test_loader.dataset)}")
for batch_inputs, batch_targets in train_loader:
    print(f"Batch Inputs Shape: {batch_inputs.shape}, Batch Targets Shape: {batch_targets.shape}")
    print(f"Example Inputs: {batch_inputs[0]}")
    print(f"Example Target: {batch_targets[0]}")
    break

Train size: 239307
Validation size: 51280
Test size: 51281
Batch Inputs Shape: torch.Size([256, 9]), Batch Targets Shape: torch.Size([256])
Example Inputs: tensor([29441,  2617,  4382, 28708,  4382,  3140,  4382, 11668, 26877])
Example Target: 20868


In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embedding_dim, num_heads, dropout=0.1):
        super(MultiHeadSelfAttention, self).__init__()
        self.num_heads = num_heads
        self.head_dim = embedding_dim // num_heads

        assert embedding_dim % num_heads == 0, "Embedding dimension must be divisible by number of heads"

        self.query = nn.Linear(embedding_dim, embedding_dim)
        self.key = nn.Linear(embedding_dim, embedding_dim)
        self.value = nn.Linear(embedding_dim, embedding_dim)

        self.out = nn.Linear(embedding_dim, embedding_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, N, E = x.size()  

        Q = self.query(x).view(B, N, self.num_heads, self.head_dim).transpose(1, 2)
        K = self.key(x).view(B, N, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.value(x).view(B, N, self.num_heads, self.head_dim).transpose(1, 2)

        scores = torch.matmul(Q, K.transpose(-2, -1)) / (self.head_dim ** 0.5)
        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)

        output = torch.matmul(attn, V)
        output = output.transpose(1, 2).contiguous().view(B, N, E)

        return self.out(output)

class PointWiseFeedForward(nn.Module):
    def __init__(self, embedding_dim, ff_dim, dropout=0.1):
        super(PointWiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(embedding_dim, ff_dim)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(ff_dim, embedding_dim)

    def forward(self, x):
        x = self.dropout(F.relu(self.linear1(x)))
        return self.linear2(x)

class CustomTransformerEncoderLayer(nn.Module):
    def __init__(self, embedding_dim, num_heads, ff_dim, dropout=0.1):
        super(CustomTransformerEncoderLayer, self).__init__()
        self.msa = MultiHeadSelfAttention(embedding_dim, num_heads, dropout)
        self.ffn = PointWiseFeedForward(embedding_dim, ff_dim, dropout)

        self.norm1 = nn.LayerNorm(embedding_dim)
        self.norm2 = nn.LayerNorm(embedding_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = x + self.dropout(self.msa(self.norm1(x)))
        x = x + self.dropout(self.ffn(self.norm2(x)))
        return x

class ModelAugmentation(nn.Module):
    def __init__(self, embedding_dim, dropout=0.1):
        super(ModelAugmentation, self).__init__()
        self.ffn = nn.Sequential(
            nn.Linear(embedding_dim, embedding_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(embedding_dim, embedding_dim)
        )
        self.norm = nn.LayerNorm(embedding_dim)

    def forward(self, x):
        return self.norm(self.ffn(x) + x)

class UET4Rec(nn.Module):
    def __init__(self, num_items, embedding_dim=128, nhead=2, max_len=9, dropout=0.2, ff_dim=256):
        super(UET4Rec, self).__init__()

        self.item_embedding = nn.Embedding(num_items, embedding_dim, padding_idx=0)

        self.position_embedding = nn.Embedding(max_len, embedding_dim)

        self.encoder1 = nn.Sequential(
            nn.Conv1d(embedding_dim, embedding_dim // 2, kernel_size=5, padding=2),
            nn.BatchNorm1d(embedding_dim // 2),
            nn.LeakyReLU(0.1),
            nn.Dropout(dropout)
        )

        self.encoder2 = nn.Sequential(
            nn.Conv1d(embedding_dim // 2, embedding_dim // 4, kernel_size=5, padding=2),
            nn.BatchNorm1d(embedding_dim // 4),
            nn.LeakyReLU(0.1),
            nn.Dropout(dropout)
        )

        self.encoder3 = nn.Sequential(
            nn.Conv1d(embedding_dim // 4, embedding_dim // 8, kernel_size=5, padding=2)
        )

        self.transformer = nn.ModuleList([
            CustomTransformerEncoderLayer(embedding_dim // 8, nhead, ff_dim, dropout) for _ in range(3)
        ])

        self.decoder1 = nn.Sequential(
            nn.Conv1d(embedding_dim // 8, embedding_dim // 4, kernel_size=5, padding=2),
            nn.BatchNorm1d(embedding_dim // 4),
            nn.LeakyReLU(0.1),
            nn.Dropout(dropout)
        )

        self.decoder2 = nn.Sequential(
            nn.Conv1d(embedding_dim // 4, embedding_dim // 2, kernel_size=5, padding=2),
            nn.BatchNorm1d(embedding_dim // 2),
            nn.LeakyReLU(0.1),
            nn.Dropout(dropout)
        )

        self.decoder3 = nn.Sequential(
            nn.Conv1d(embedding_dim // 2, embedding_dim, kernel_size=5, padding=2)
        )

        self.ma = ModelAugmentation(embedding_dim, dropout)

        self.fc = nn.Linear(embedding_dim, num_items)

    def forward(self, x):
        batch_size, seq_len = x.size()

        positions = torch.arange(0, seq_len).unsqueeze(0).repeat(batch_size, 1).to(x.device)
        x = self.item_embedding(x) + self.position_embedding(positions)

        x = x.permute(0, 2, 1)  
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(enc1)
        enc3 = self.encoder3(enc2)

        x = enc3.permute(0, 2, 1)  
        for layer in self.transformer:
            x = layer(x)
        x = x.permute(0, 2, 1)  

        dec1 = self.decoder1(x + enc3)  
        dec2 = self.decoder2(dec1 + enc2)  
        dec3 = self.decoder3(dec2 + enc1)  

        x = dec3[:, :, -1]  
        x = self.ma(x)  
        out = self.fc(x)  

        return out
embedding_dim = 128
nhead = 2
ff_dim = 256
dropout = 0.7
max_len = 9

model = UET4Rec(
    num_items=num_items,
    embedding_dim=embedding_dim,
    nhead=nhead,
    max_len=max_len,
    dropout=dropout,
    ff_dim=ff_dim
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(model)

UET4Rec(
  (item_embedding): Embedding(31228, 128, padding_idx=0)
  (position_embedding): Embedding(9, 128)
  (encoder1): Sequential(
    (0): Conv1d(128, 64, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1)
    (3): Dropout(p=0.7, inplace=False)
  )
  (encoder2): Sequential(
    (0): Conv1d(64, 32, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1)
    (3): Dropout(p=0.7, inplace=False)
  )
  (encoder3): Sequential(
    (0): Conv1d(32, 16, kernel_size=(5,), stride=(1,), padding=(2,))
  )
  (transformer): ModuleList(
    (0-2): 3 x CustomTransformerEncoderLayer(
      (msa): MultiHeadSelfAttention(
        (query): Linear(in_features=16, out_features=16, bias=True)
        (key): Linear(in_features=16, out_features=16, bias=True)
        (value

In [24]:
import torch
import torch.optim as optim
from tqdm import tqdm
import numpy as np
from sklearn.metrics import ndcg_score


def evaluate_hit_ndcg_multi_k(model, val_loader, ks=[5, 10, 20]):
    model.eval()
    total_hr = {k: 0 for k in ks}
    total_ndcg = {k: 0 for k in ks}
    total_samples = 0

    with torch.no_grad():
        for batch_inputs, batch_targets in tqdm(val_loader, desc="Evaluating HR & NDCG for multiple k"):
            batch_inputs = batch_inputs.to(device)
            batch_targets = batch_targets.to(device)
            outputs = model(batch_inputs)  # Forward pass
            _, top_k_indices = torch.topk(outputs, k=max(ks), dim=1)  # Get top-k indices

            for i in range(batch_targets.size(0)):
                target = batch_targets[i].item()
                predictions = top_k_indices[i].cpu().numpy()

                for k in ks:
                    top_k_preds = predictions[:k]

                    if target in top_k_preds:
                        total_hr[k] += 1

                    if target in top_k_preds:
                        rank = np.where(top_k_preds == target)[0][0] + 1
                        total_ndcg[k] += 1 / np.log2(rank + 1)

            total_samples += batch_targets.size(0)

    hr = {k: total_hr[k] / total_samples for k in ks}
    ndcg = {k: total_ndcg[k] / total_samples for k in ks}

    return hr, ndcg

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 1000
save_interval = 50
ks = [5, 10, 20]  

print("Starting Training...")

for epoch in range(1, num_epochs + 1):
    model.train()
    total_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    with tqdm(train_loader, desc=f"Epoch [{epoch}/{num_epochs}]", unit="batch") as progress_bar:
        for batch_inputs, batch_targets in progress_bar:
            batch_inputs, batch_targets = batch_inputs.to(device), batch_targets.to(device)

            optimizer.zero_grad()

            outputs = model(batch_inputs)  
            loss = criterion(outputs, batch_targets)  

            loss.backward()  
            optimizer.step()  

            total_loss += loss.item()

            _, predicted = torch.max(outputs, dim=1)
            correct_predictions += (predicted == batch_targets).sum().item()
            total_predictions += batch_targets.size(0)

            progress_bar.set_postfix({"Batch Loss": loss.item()})

    avg_loss = total_loss / len(train_loader)
    accuracy = correct_predictions / total_predictions

    print(f"Epoch [{epoch}/{num_epochs}], Training Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

    if epoch % save_interval == 0:
        print("\nEvaluating on validation set...")
        hr, ndcg = evaluate_hit_ndcg_multi_k(model, val_loader, ks=ks)
        for k in ks:
            print(f"Validation HR@{k}: {hr[k]:.4f}, NDCG@{k}: {ndcg[k]:.4f}")

        model_save_path = f"uet4rec_model_epoch_{epoch}.pth"
        torch.save(model.state_dict(), model_save_path)
        print(f"Model saved at '{model_save_path}'")

final_model_path = "uet4rec_model_final.pth"
torch.save(model.state_dict(), final_model_path)
print(f"Final model saved as '{final_model_path}'")

print("Training Completed.")


Starting Training...


Epoch [1/1000]: 100%|██████████| 935/935 [00:15<00:00, 59.00batch/s, Batch Loss=9.89]


Epoch [1/1000], Training Loss: 9.9927, Accuracy: 0.0030


Epoch [2/1000]: 100%|██████████| 935/935 [00:15<00:00, 61.89batch/s, Batch Loss=8.82]


Epoch [2/1000], Training Loss: 9.2475, Accuracy: 0.0324


...

Epoch [992/1000]: 100%|██████████| 935/935 [00:14<00:00, 64.46batch/s, Batch Loss=0.936]


Epoch [992/1000], Training Loss: 0.8232, Accuracy: 0.7767


Epoch [993/1000]: 100%|██████████| 935/935 [00:14<00:00, 64.79batch/s, Batch Loss=0.758]


Epoch [993/1000], Training Loss: 0.8261, Accuracy: 0.7770


Epoch [994/1000]:  61%|██████    | 566/935 [00:08<00:05, 64.78batch/s, Batch Loss=0.809]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [27]:
import torch
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm


In [29]:
import numpy as np
import torch
from tqdm import tqdm
from sklearn.metrics import ndcg_score

model = UET4Rec(
    num_items=num_items,
    embedding_dim=embedding_dim,
    nhead=nhead,
    max_len=max_len,
    dropout=dropout,
    ff_dim=ff_dim
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load("uet4rec_model_final.pth", map_location=device)) 
model.to(device)
model.eval() 
print("Model loaded successfully.")


def evaluate_hit_ndcg_multi_k(model, test_loader, ks=[5, 10, 20]):
    model.eval()
    total_hr = {k: 0 for k in ks}
    total_ndcg = {k: 0 for k in ks}
    total_samples = 0

    with torch.no_grad():
        for batch_inputs, batch_targets in tqdm(test_loader, desc="Evaluating HR & NDCG for multiple k"):
            batch_inputs = batch_inputs.to(device)
            batch_targets = batch_targets.to(device)
            outputs = model(batch_inputs)  
            _, top_k_indices = torch.topk(outputs, k=max(ks), dim=1) 

            for i in range(batch_targets.size(0)):
                target = batch_targets[i].item()
                predictions = top_k_indices[i].cpu().numpy()

                for k in ks:
                    top_k_preds = predictions[:k]

                    if target in top_k_preds:
                        total_hr[k] += 1

                    if target in top_k_preds:
                        rank = np.where(top_k_preds == target)[0][0] + 1
                        total_ndcg[k] += 1 / np.log2(rank + 1)

            total_samples += batch_targets.size(0)

    hr = {k: total_hr[k] / total_samples for k in ks}
    ndcg = {k: total_ndcg[k] / total_samples for k in ks}

    return hr, ndcg

k_values = [5, 10, 20]
hr, ndcg = evaluate_hit_ndcg_multi_k(model, test_loader, ks=k_values)

print("\nMetrics Summary:")
for k in k_values:
    print(f"HR@{k}: {hr[k]:.4f}")
    print(f"NDCG@{k}: {ndcg[k]:.4f}")


Model loaded successfully.


Evaluating HR & NDCG for multiple k: 100%|██████████| 201/201 [00:05<00:00, 38.70it/s]


Metrics Summary:
HR@5: 0.3735
NDCG@5: 0.3068
HR@10: 0.4147
NDCG@10: 0.3202
HR@20: 0.4515
NDCG@20: 0.3295


In [32]:
df.head()

,Timestamp,UserId,event,ProductId,transactionid,reward
0,2015-08-07 17:51:44.567,1,view,21761,NaN,0.2
1,2015-08-07 17:53:33.790,1,view,21761,NaN,0.2
2,2015-08-07 17:56:52.664,1,view,17456,NaN,0.2
3,2015-08-07 18:01:08.920,1,view,14493,NaN,0.2
4,2015-08-07 18:08:25.669,1,view,22900,NaN,0.2
